In [13]:
%load_ext autoreload
%autoreload 2
import pickle as pkl
import warnings

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.base import ClassifierMixin, RegressorMixin

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
from collections import defaultdict
import sys
import joblib
import dvu
dvu.set_style()
sys.path.append('../results')
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import auggam.config
RESULTS_DIR = join(auggam.config.results_dir, '7gram')

In [15]:
fnames = [fname for fname in os.listdir(RESULTS_DIR)] # if fname.startswith('instructor')]
# accs = pd.read_pickle('../results/instructor_accs.pkl')

In [16]:
fnames_acc = [fname for fname in fnames if 'acc_' in fname]

In [25]:
df = defaultdict(list)
def rename_bert_models(s):
    if 'bert' in s and not s == 'bert-base-uncased':
        return 'bert-finetuned'
    else:
        return s

for fname in fnames_acc:
    # print(fname)
    acc = joblib.load(join(RESULTS_DIR, fname))
    # print(acc)
    df['acc'].append(acc['acc_val'])
    # print(fname.split('_acc_'))
    df['model'].append(fname.split('_acc_')[0])
    df['dset'].append(fname.split('_acc_')[1].replace('_imodelsx.pkl', ''))
# joblib.load(join('../results', fnames_acc[0]))
df = pd.DataFrame(df)
df['model'] = df['model'].apply(rename_bert_models)
df

,acc,model,dset
0,0.800188,instructor,rotten_tomatoes
1,0.496000,gpt2,emotion
2,0.819887,llama_7b,rotten_tomatoes
3,0.709500,instructor,emotion
4,0.879079,tfidfvectorizer,ag_news
5,0.760526,instructor,financial_phrasebank
6,0.874000,tfidfvectorizer,trec
7,0.761726,gpt2,rotten_tomatoes
8,0.648500,linear_finetune,emotion
9,0.863158,linear_finetune,financial_phrasebank


In [26]:
df.pivot(index='model', columns='dset', values='acc')

dset,ag_news,dbpedia_14,emotion,financial_phrasebank,rotten_tomatoes,sst2,trec
model,,,,,,,
bert-base-uncased,0.887763,NaN,NaN,NaN,NaN,NaN,0.890
bert-finetuned,0.906447,NaN,NaN,NaN,NaN,NaN,0.944
gpt2,0.861053,NaN,0.4960,0.719298,0.761726,0.714450,NaN
gpt2-xl,NaN,NaN,0.6805,0.759649,0.815197,0.787844,NaN
instructor,NaN,NaN,0.7095,0.760526,0.800188,0.847477,NaN
linear_finetune,NaN,0.983714,0.6485,0.863158,0.813321,0.853211,NaN
llama_7b,NaN,NaN,0.7335,0.821053,0.819887,0.816514,NaN
tfidfvectorizer,0.879079,0.956829,0.8600,0.833333,0.749531,0.810780,0.874
